## Deliverable 2. Create a Customer Travel Destinations Map.

In [1]:
# Dependencies and Setup
import pandas as pd
import requests
import gmaps
import os
import sys
import numpy as np

# Import API key
from config import g_key


# Configure gmaps API key
gmaps.configure(api_key=g_key)

In [2]:
# 1. Import the WeatherPy_database.csv file. 
city_data_df = pd.read_csv("Weather_Database/WeatherPy_database.csv")
city_data_df.head()

,City_ID,City,Country,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Current Description
0,0,Cidreira,BR,-30.1811,-50.2056,71.58,74,0,16.20,clear sky
1,1,Santa Cruz Del Sur,CU,20.7147,-77.9956,85.68,67,100,15.57,overcast clouds
2,2,Barrow,US,71.2906,-156.7887,23.02,86,90,11.50,light snow
3,3,Saint-Philippe,RE,-21.3585,55.7679,74.79,78,2,9.13,clear sky
4,4,Sakaiminato,JP,35.5500,133.2333,60.78,51,75,14.97,broken clouds


In [3]:
# 2. Prompt the user to enter minimum and maximum temperature criteria 
min_temp = float(input("What is the minimum temperature you would like for your trip? "))
max_temp = float(input("What is the maximum temperature you would like for your trip? "))

What is the minimum temperature you would like for your trip? 75
What is the maximum temperature you would like for your trip? 90


In [4]:
# 3. Filter the city_data_df DataFrame using the input statements to create a new DataFrame using the loc method.
preferred_cities_df = city_data_df.loc[(city_data_df["Max Temp"] <= max_temp) & \
                                       (city_data_df["Max Temp"] >= min_temp)]
preferred_cities_df.head(10)

,City_ID,City,Country,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Current Description
1,1,Santa Cruz Del Sur,CU,20.7147,-77.9956,85.68,67,100,15.57,overcast clouds
10,10,Lubao,PH,14.9167,120.5500,80.15,83,60,8.72,broken clouds
11,11,Bambous Virieux,MU,-20.3428,57.7575,75.22,83,20,10.36,few clouds
17,17,Lorengau,PG,-2.0226,147.2712,81.27,78,100,4.85,overcast clouds
18,18,Namatanai,PG,-3.6667,152.4333,80.44,78,97,7.20,overcast clouds
21,21,Chuy,UY,-33.6971,-53.4616,76.66,53,0,11.90,clear sky
27,27,Kununurra,AU,-15.7667,128.7333,80.58,78,40,6.91,scattered clouds
29,29,Hithadhoo,MV,-0.6000,73.0833,83.16,75,100,15.66,overcast clouds
31,31,Rikitea,PF,-23.1203,-134.9692,75.20,81,96,13.11,light rain
32,32,Arraial Do Cabo,BR,-22.9661,-42.0278,76.96,83,75,13.80,broken clouds


In [5]:
# 4a. Determine if there are any empty rows.
preferred_cities_df.count()

City_ID                202
City                   202
Country                201
Lat                    202
Lng                    202
Max Temp               202
Humidity               202
Cloudiness             202
Wind Speed             202
Current Description    202
dtype: int64

In [6]:
# 4b. Drop any empty rows and create a new DataFrame that doesn’t have empty rows.
clean_df = preferred_cities_df.dropna()
clean_df.count()

City_ID                201
City                   201
Country                201
Lat                    201
Lng                    201
Max Temp               201
Humidity               201
Cloudiness             201
Wind Speed             201
Current Description    201
dtype: int64

In [7]:
# 5a. Create DataFrame called hotel_df to store hotel names along with city, country, max temp, and coordinates.
hotel_df = clean_df[["City", "Country", "Max Temp", "Current Description", "Lat", "Lng"]].copy()

# 5b. Create a new column "Hotel Name"
hotel_df["Hotel Name"] = ""
hotel_df.head(10)

,City,Country,Max Temp,Current Description,Lat,Lng,Hotel Name
1,Santa Cruz Del Sur,CU,85.68,overcast clouds,20.7147,-77.9956,
10,Lubao,PH,80.15,broken clouds,14.9167,120.5500,
11,Bambous Virieux,MU,75.22,few clouds,-20.3428,57.7575,
17,Lorengau,PG,81.27,overcast clouds,-2.0226,147.2712,
18,Namatanai,PG,80.44,overcast clouds,-3.6667,152.4333,
21,Chuy,UY,76.66,clear sky,-33.6971,-53.4616,
27,Kununurra,AU,80.58,scattered clouds,-15.7667,128.7333,
29,Hithadhoo,MV,83.16,overcast clouds,-0.6000,73.0833,
31,Rikitea,PF,75.20,light rain,-23.1203,-134.9692,
32,Arraial Do Cabo,BR,76.96,broken clouds,-22.9661,-42.0278,


In [8]:
# 6a. Set parameters to search for hotels with 5000 meters.
params = {
    "radius": 5000,
    "type": "lodging",
    "key": g_key
}

# 6b. Iterate through the hotel DataFrame.
for index, row in hotel_df.iterrows():
    # 6c. Get latitude and longitude from DataFrame
    lat = row["Lat"]
    lng = row["Lng"]
    
    params["location"] = f"{lat},{lng}"
    
    # 6d. Set up the base URL for the Google Directions API to get JSON data.
    base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"

    # 6e. Make request and retrieve the JSON data from the search. 
    hotels = requests.get(base_url, params=params).json()
    
    # 6f. Get the first hotel from the results and store the name, if a hotel isn't found skip the city.
    try:
        hotel_df.loc[index, "Hotel Name"] = hotels["results"][0]["name"]
    except (IndexError):
        print("Hotel not found... skipping.")
        

Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.


In [9]:
hotel_df['Hotel Name'].replace('', np.nan, inplace=True)

In [10]:
# 7. Drop the rows where there is no Hotel Name.
clean_hotel_df = hotel_df.dropna()

In [11]:
clean_hotel_df.head(10)

,City,Country,Max Temp,Current Description,Lat,Lng,Hotel Name
10,Lubao,PH,80.15,broken clouds,14.9167,120.5500,Hacienda Gracia
11,Bambous Virieux,MU,75.22,few clouds,-20.3428,57.7575,Casa Tia Villa
17,Lorengau,PG,81.27,overcast clouds,-2.0226,147.2712,Lorengau Harbourside Hotel
18,Namatanai,PG,80.44,overcast clouds,-3.6667,152.4333,Seagulls Inn Namatani
21,Chuy,UY,76.66,clear sky,-33.6971,-53.4616,Nuevo Hotel Plaza
27,Kununurra,AU,80.58,scattered clouds,-15.7667,128.7333,Hotel Kununurra
29,Hithadhoo,MV,83.16,overcast clouds,-0.6000,73.0833,Scoop Guest House
31,Rikitea,PF,75.20,light rain,-23.1203,-134.9692,People ThankYou
32,Arraial Do Cabo,BR,76.96,broken clouds,-22.9661,-42.0278,Pousada Porto Praia
34,Gizo,SB,82.63,scattered clouds,-8.1030,156.8419,Imagination Island


In [12]:
# 8a. Create the output File (CSV)
output_data_file = "WeatherPy_vacation.csv"
# 8b. Export the City_Data into a csv
clean_hotel_df.to_csv(output_data_file, index_label="City_ID")

In [13]:
# 9. Using the template add city name, the country code, the weather description and maximum temperature for the city.
info_box_template = """
<dl>
<dt>Hotel Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
<dt>Max Temp</dt><dd>{Max Temp} °F</dd>
</dl>
"""

# 10a. Get the data from each row and add it to the formatting template and store the data in a list.
hotel_info = [info_box_template.format(**row) for index, row in clean_hotel_df.iterrows()]

# 10b. Get the latitude and longitude from each row and store in a new DataFrame.
locations = clean_hotel_df[["Lat", "Lng"]]

In [14]:
# 11a. Add a marker layer for each city to the map. 
locations = clean_hotel_df[["Lat", "Lng"]]
max_temp = clean_hotel_df["Max Temp"]
fig = gmaps.figure(center=(30.0, 31.0), zoom_level=1.5)
heat_layer = gmaps.heatmap_layer(locations, weights=max_temp,dissipating=False,
             max_intensity=300, point_radius=4)
marker_layer = gmaps.marker_layer(locations, info_box_content=hotel_info)
fig.add_layer(heat_layer)
fig.add_layer(marker_layer)

# 11b. Display the figure
fig

Figure(layout=FigureLayout(height='420px'))